In [94]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, struct
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, TimestampType, DoubleType




In [95]:
spark = SparkSession.builder.getOrCreate()
spark = SparkSession.builder.appName("API Joining").getOrCreate()


In [96]:
account_url = "https://xloop-dummy.herokuapp.com/account"
councillor_url = "https://xloop-dummy.herokuapp.com/councillor"
patient_url = "https://xloop-dummy.herokuapp.com/patient"


In [97]:
account_data = requests.get(account_url).json()
councillor_data = requests.get(councillor_url).json()
patient_data = requests.get(patient_url).json()

In [100]:
schema = StructType([
    StructField("id", IntegerType(), nullable=False),
    StructField("created", StringType(), nullable=False),
    StructField("updated", StringType(), nullable=False),
    StructField("email", StringType(), nullable=False),
    StructField("password", StringType(), nullable=False),
    StructField("first_name", StringType(), nullable=False),
    StructField("last_name", StringType(), nullable=False),
    StructField("gender", StringType(), nullable=False),
    StructField("phone_number", StringType(), nullable=False),
    StructField("address", StructType([
        StructField("address", StringType(), nullable=False),
        StructField("location", StructType([
            StructField("lat", DoubleType(), nullable=False),
            StructField("lng", DoubleType(), nullable=False)
        ])),
        StructField("placeId", StringType(), nullable=False),
        StructField("region", StringType(), nullable=False)
    ])),
    StructField("national_identity", StringType(), nullable=False),
    StructField("role", StringType(), nullable=False),
    StructField("is_active", StringType(), nullable=False)
])


In [101]:
account_df = spark.createDataFrame(account_data,schema)
councillor_df = spark.createDataFrame(councillor_data)
patient_df = spark.createDataFrame(patient_data)


In [102]:
account_df.printSchema()

root
 |-- id: integer (nullable = false)
 |-- created: string (nullable = false)
 |-- updated: string (nullable = false)
 |-- email: string (nullable = false)
 |-- password: string (nullable = false)
 |-- first_name: string (nullable = false)
 |-- last_name: string (nullable = false)
 |-- gender: string (nullable = false)
 |-- phone_number: string (nullable = false)
 |-- address: struct (nullable = true)
 |    |-- address: string (nullable = false)
 |    |-- location: struct (nullable = true)
 |    |    |-- lat: double (nullable = false)
 |    |    |-- lng: double (nullable = false)
 |    |-- placeId: string (nullable = false)
 |    |-- region: string (nullable = false)
 |-- national_identity: string (nullable = false)
 |-- role: string (nullable = false)
 |-- is_active: string (nullable = false)



In [103]:
location_schema = struct(
    col("address.location.lat").alias("lat"),
    col("address.location.lng").alias("lng")
)

In [104]:
selected_df = account_df.select("id", "address.location", "address.region")
selected_df.show(truncate=False)

+----+----------------------------------------+------+
|id  |location                                |region|
+----+----------------------------------------+------+
|4481|{37.83735852005235, 61.21006971841348}  |PK    |
|2344|{34.513027260782934, 75.29883307124692} |PK    |
|2308|{26.67471399032989, 72.97968890693005}  |PK    |
|9968|{23.22676257565109, 68.0226556244778}   |PK    |
|2874|{15.106643140881266, 62.97671373513003} |PK    |
|6598|{25.37761993327233, 65.95173619970035}  |PK    |
|4404|{20.816752450682387, 74.35004023317276} |PK    |
|1444|{24.670211916430944, 57.85636615736132} |PK    |
|3118|{28.34459082856146, 64.0995373298432}   |PK    |
|561 |{27.7340023307804, 77.60132569940936}   |PK    |
|9719|{19.730077922311708, 63.437169265474495}|PK    |
|7391|{26.51228866336421, 69.58286254834991}  |PK    |
|9082|{28.253405671726814, 57.62803793273429} |PK    |
|7707|{26.56883668885961, 53.58757938984559}  |PK    |
|9146|{29.894270246926105, 64.96318136156651} |PK    |
|230 |{12.

In [105]:
selected_df = account_df.select(col("id"), col("address.region"),col("address.location.lat"), col("address.location.lng"))

# Show the selected data
selected_df.show(truncate=False)

+----+------+------------------+------------------+
|id  |region|lat               |lng               |
+----+------+------------------+------------------+
|4481|PK    |37.83735852005235 |61.21006971841348 |
|2344|PK    |34.513027260782934|75.29883307124692 |
|2308|PK    |26.67471399032989 |72.97968890693005 |
|9968|PK    |23.22676257565109 |68.0226556244778  |
|2874|PK    |15.106643140881266|62.97671373513003 |
|6598|PK    |25.37761993327233 |65.95173619970035 |
|4404|PK    |20.816752450682387|74.35004023317276 |
|1444|PK    |24.670211916430944|57.85636615736132 |
|3118|PK    |28.34459082856146 |64.0995373298432  |
|561 |PK    |27.7340023307804  |77.60132569940936 |
|9719|PK    |19.730077922311708|63.437169265474495|
|7391|PK    |26.51228866336421 |69.58286254834991 |
|9082|PK    |28.253405671726814|57.62803793273429 |
|7707|PK    |26.56883668885961 |53.58757938984559 |
|9146|PK    |29.894270246926105|64.96318136156651 |
|230 |PK    |12.229550431509992|67.53370483725337 |
|17  |PK    

In [109]:
selected_df = account_df.select(col("id").alias("user_id"), col("address.region"), col("address.location.lat"), col("address.location.lng"))
patient_selected_df = patient_df.select("user_id", "id")
selected_df.show()

+-------+------+------------------+------------------+
|user_id|region|               lat|               lng|
+-------+------+------------------+------------------+
|   4481|    PK| 37.83735852005235| 61.21006971841348|
|   2344|    PK|34.513027260782934| 75.29883307124692|
|   2308|    PK| 26.67471399032989| 72.97968890693005|
|   9968|    PK| 23.22676257565109|  68.0226556244778|
|   2874|    PK|15.106643140881266| 62.97671373513003|
|   6598|    PK| 25.37761993327233| 65.95173619970035|
|   4404|    PK|20.816752450682387| 74.35004023317276|
|   1444|    PK|24.670211916430944| 57.85636615736132|
|   3118|    PK| 28.34459082856146|  64.0995373298432|
|    561|    PK|  27.7340023307804| 77.60132569940936|
|   9719|    PK|19.730077922311708|63.437169265474495|
|   7391|    PK| 26.51228866336421| 69.58286254834991|
|   9082|    PK|28.253405671726814| 57.62803793273429|
|   7707|    PK| 26.56883668885961| 53.58757938984559|
|   9146|    PK|29.894270246926105| 64.96318136156651|
|    230| 

In [110]:
joined_df = selected_df.join(patient_selected_df, selected_df.user_id == patient_selected_df.user_id)
joined_df.show()

+-------+------+------------------+------------------+-------+----+
|user_id|region|               lat|               lng|user_id|  id|
+-------+------+------------------+------------------+-------+----+
|    418|    PK| 24.62339084725973| 73.85793958902661|    418|6088|
|   6892|    PK| 25.46743322510808| 72.99090456101771|   6892|2657|
|   6892|    PK| 25.45984206410578| 79.94908847679721|   6892|2657|
|   2895|    PK|23.215602269191226| 67.87813407557383|   2895|2747|
|   3061|    PK|28.468373832239926| 75.33444503272551|   3061|5399|
|   3061|    PK|21.137165903823394| 64.45490124652406|   3061|5399|
|   3061|    PK|25.549517067785036|53.723753666017956|   3061|5399|
|   5148|    PK|14.258286500332598| 67.34708427163662|   5148|3829|
|   5148|    PK|26.060769033421174| 57.24422958355194|   5148|3829|
|   7664|    PK|28.941385642841272| 68.99977916680939|   7664|2975|
|    222|    PK| 24.46227849980198|  66.3273700471616|    222|5395|
|   6156|    PK| 38.01377915863122| 67.127456663

In [108]:
joined_df_1 = selected_df.join(councillor_selected_df, selected_df.account_id == councillor_selected_df.user_id)
joined_df_1.show()

+----------+------+------------------+------------------+-------+----+
|account_id|region|               lat|               lng|user_id|  id|
+----------+------+------------------+------------------+-------+----+
|      2927|    PK| 20.07127106936702| 72.07580882526509|   2927|9510|
|      1950|    PK| 37.68087486742396| 67.78649977152259|   1950| 824|
|      3091|    PK|30.514693025039854|  72.2841645677654|   3091|8955|
|      8209|    PK| 32.68559601994396| 72.73932690021184|   8209| 884|
|      1360|    PK| 19.79106789068872|   59.871373119959|   1360| 686|
|      6856|    PK|22.633145515602198| 62.56751192226511|   6856|2376|
|      4191|    PK| 18.90469668967994| 76.19573981799499|   4191|3248|
|      7146|    PK|28.778919286785374| 54.50485101969218|   7146|7164|
|      7146|    PK| 33.09941925439517| 73.13091562725617|   7146|7164|
|      6700|    PK|26.515990765547105|    76.64585872055|   6700|2844|
|      6319|    PK| 16.68899592329044|60.981833174537655|   6319|5466|
|     

In [32]:
councillor_selected_df = councillor_df.select("user_id", "id")
patient_selected_df = patient_df.select("user_id", "id")

In [40]:
joined_df = councillor_selected_df.join(patient_selected_df, "user_id")

In [41]:
joined_df.show()

+-------+----+----+
|user_id|  id|  id|
+-------+----+----+
|   6834|6738|3158|
|   6834|9625|3158|
|   4126|1890|9018|
|   4126|5232|9018|
|   6892|9911|2657|
|   3061|3526|5399|
|   5148|3797|3829|
|   7664|1730|2975|
|   3741|2951|8776|
|   2570|9619|8987|
|   2517|2821|1564|
|   6489|3864|4803|
|   8872|4473|8002|
|   8872|8810|8002|
|   9686| 360|3289|
|   9686|7102|3289|
|   9686|3653|3289|
|   9686|5730|3289|
|   1447|9235|6768|
|   1447|1139|6768|
+-------+----+----+
only showing top 20 rows



In [ ]:
councillor_selected_df = councillor_df.select("user_id", "id")
patient_selected_df = patient_df.select("user_id", "id")


In [6]:
joined_df = account_df.join(councillor_df, account_df["common_column"] == councillor_df["common_column"], "inner") \
                      .join(patient_df, account_df["common_column"] == patient_df["common_column"], "inner")


In [8]:
joined_df = councillor_selected_df.join(patient_selected_df, "user_id")


DataFrame[created: string, guardian_phone_number: string, id: bigint, updated: string, user_id: bigint]

In [ ]:
selected_df = account_df.select(col("id").alias("account_id"), col("address.region"), col("address.location.lat"), col("address.location.lng"))
patient_selected_df = patient_df.select("user_id", "id")

# Join the DataFrames on the common column
joined_df = selected_df.join(patient_selected_df, selected_df.account_id == patient_selected_df.user_id)

# Show the result